## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [ ]:
import requests

In [ ]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json() 

In [ ]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [ ]:
len(data)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.head(5)

In [ ]:
# We drop the 'rental methods' column, as it contains multiple values and 
# we cannot insert lists in a database cell.
df.drop('rental_methods', axis=1, inplace = True)
df.head(5)

In [ ]:
df['rental_url'][0]

### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [ ]:
!sudo -H pip3 install -U sqlalchemy

In [ ]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = 'localhost', 
    user = 'root',
    password = 'dwdstudent2015')

engine = create_engine(conn_string)
con = engine.connect()

Once we have connected successfully, we need to create our database:

In [ ]:
# Query to create a database
db_name = 'citibike_stations'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
engine.execute(create_db_query)
# And lets switch to the database
engine.execute("USE {db}".format(db=db_name))

Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [ ]:
# This step is typically optional, but it is good practice to define explicitly the 
# data types before storing things in a database. In many cases, this can be ommitted, though.
import sqlalchemy
dtype = {
    'capacity': sqlalchemy.types.SMALLINT(),
    'eightd_has_key_dispenser':  sqlalchemy.types.BOOLEAN,
    'lat': sqlalchemy.types.Float, 
    'lon': sqlalchemy.types.Float,
    'name': sqlalchemy.types.VARCHAR(50),
    'region_id': sqlalchemy.types.VARCHAR(5),
    'rental_url': sqlalchemy.types.VARCHAR(100),
    'short_name': sqlalchemy.types.VARCHAR(10),
    'station_id': sqlalchemy.types.SMALLINT()
}

In [ ]:
table_name = 'Stations'
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
df.to_sql(name = table_name, 
          schema = db_name, 
          con = engine, 
          if_exists = 'replace', 
          index = False, 
          dtype = dtype)

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = 'ALTER TABLE Stations ADD PRIMARY KEY(station_id)'
engine.execute(add_key_query)

### Reading from a SQL Database in Python using Pandas

We can similarly read from the database using Pandas

In [ ]:
query =  "SELECT * FROM Stations"
df2 = pd.read_sql(query, con=engine)
df2.head(5)

And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [ ]:
# The necessary library to write in Excel
!sudo pip3 install -U xlwt

In [ ]:
df2.to_excel("citibike.xls")
df2.to_csv("citibike.csv")

### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.